In [1]:
%matplotlib inline
import networkx as nx

<div class="alert alert-info">
<center>
  <h1>Programación Dinamica</h1>
</center>
</div>

<div class="alert alert-success" style="width:90%; margin:0 auto;">

  <h2><p>1 - Distancia de edición</p></h2>
  
  <p> Dadas dos cadenas S1 y S2, este algoritmo trata de encontrar la "distancia" entre ellas. Basamos la distancia en el número de transformaciones necesarias para convertir la cadena S1 en la cadena S2 (o viceversa). Se consideran tres transformaciones distintas: inserción, eliminación y sustitución. Los costes de cada operación serán parámetros de la función.</p>
  
  <img src='img/min_Edit_Distance_DP_Table.png' width="50%">
  
Además, se os pide que reconstruyáis el conjunto de operaciones que realizar sobre la primera palabra (<i>relevant</i>) para convertirla en la segunda (<i>elephant</i>). En el ejemplo de la imagen (donde los costes de las operaciones es 1 para todas):
<ul>
    <li> Eliminar 'R' en la posición 0.           $('D','R',0)$</li>
    <li> Insertar 'P' en la posición 4.           $('I','P',4)$</li>
    <li> Substituir 'V' por 'H' en la posición 5.            $('R',('V', 'H'),4)$ </li>
    <li> ........ El coste total de edición es: 3 </li>
</ul>

** Nota: Vuestro algoritmo podría encontrar otro conjunto de operaciones distinto a este pero con el mismo coste de edición.

In [18]:
import numpy as np
'''
Implementa aquí el algoritmo de distancia de edición
'''
def distancia_edicion(s1,s2,ci=2,cd=2,cr=1):
    m = np.zeros((len(s1)+1, len(s2)+1))
    
    for i in range(0,len(s1)+1):
        m[i][0] = i*ci
    for j in range(0,len(s2)+1):
        m[0][j] = j*ci
        
    for i in range(1,len(s1)+1):
        for j in range(1,len(s2)+1):
            if(s1[i-1] == s2[j-1]):
                m[i][j] = m[i-1][j-1]
            else:
                m[i][j] = min(m[i-1,j]+cd,m[i,j-1]+ci,m[i-1,j-1]+cr)
            
    return m

In [19]:
def distancia_edicion_clase(s1,s2,ci=2,cd=2,cr=1):
    distance_matrix =np.zeros((len(s1)+1, len(s2)+1))
    parent_matrix= [[None for _ in range(len(s2)+1)] for _ in range(len(s1)+1)]
    
    for i in range(1,len(s1)+1):
        distance_matrix[i,0] = i*cd
        parent_matrix[i][0] = "D"
        
    for j in range(1,len(s2)+1):
        distance_matrix[0,j] = j*ci
        parent_matrix[i][0] = "I"
    
    for i in range(1,len(s1)+1):
        for j in range(1,len(s2)+1):
            delete = distance_matrix[i-1,j] + cd
            insert = distance_matrix[i,j-1] + ci
            replac = distance_matrix[i-1,j-1]
            if s1[i-1] != s2[j-1]:
                replac -= cr
            distance_matrix[i,j],parent_matrix[i][j] = min((insert, "I"), (delete, "D"), (replac, "R"))
        
    
    #Get necessary operations
    pm1= len(s1)
    edit_distance,pm2 = min(zip(distance_matrix[pm1],range(len(s2)+1)))
    operations=[]
    predecessor = parent_matrix[pm1][pm2]
    while predecessor:
        if predecessor == "I":
            operations.append((predecesor,s2[pm2-1],pm-1))
            pm2-=1
        elif predecessor == "D":
            operations.append()
            pm1 -=1
        elif predecessor == "R":
            if s1[pm1-1] != s2[pm2-1]:
                operations.append((predecessor,(s1[pm1-1],s2[pm2-1]),pm1-1))
            pm1-=1
            pm2-=1
        predecesor = parent_matrix[pm1][pm2]

    

In [22]:
print(distancia_edicion('relevant','elephant'))
print(distancia_edicion('surgery', 'survey'))
print(distancia_edicion('abcde', 'vwxyz'))

[[ 0.  2.  4.  6.  8. 10. 12. 14. 16.]
 [ 2.  1.  3.  5.  7.  9. 11. 13. 15.]
 [ 4.  2.  2.  3.  5.  7.  9. 11. 13.]
 [ 6.  4.  2.  3.  4.  6.  8. 10. 12.]
 [ 8.  6.  4.  2.  4.  5.  7.  9. 11.]
 [10.  8.  6.  4.  3.  5.  6.  8. 10.]
 [12. 10.  8.  6.  5.  4.  5.  7.  9.]
 [14. 12. 10.  8.  7.  6.  5.  5.  7.]
 [16. 14. 12. 10.  9.  8.  7.  6.  5.]]
[[ 0.  2.  4.  6.  8. 10. 12.]
 [ 2.  0.  2.  4.  6.  8. 10.]
 [ 4.  2.  0.  2.  4.  6.  8.]
 [ 6.  4.  2.  0.  2.  4.  6.]
 [ 8.  6.  4.  2.  1.  3.  5.]
 [10.  8.  6.  4.  3.  1.  3.]
 [12. 10.  8.  6.  5.  3.  2.]
 [14. 12. 10.  8.  7.  5.  3.]]
[[ 0.  2.  4.  6.  8. 10.]
 [ 2.  1.  3.  5.  7.  9.]
 [ 4.  3.  2.  4.  6.  8.]
 [ 6.  5.  4.  3.  5.  7.]
 [ 8.  7.  6.  5.  4.  6.]
 [10.  9.  8.  7.  6.  5.]]


<div class="alert alert-success" style="width:90%; margin:0 auto;">

  <h2><p>2 - Torres de Hanoi</p></h2>
  
  <p>Se plantea un escenario con 3 torres y una serie de discos de diferentes tamaños. Buscamos mover todos los discos de una torre origen a otra destino empleando la torre intermedia como soporte. Siempre deberán cumplirse las siguientes condiciones:
<ul>
    <li> Solo podemos mover un disco a cada vez. </li>
    <li> Un movimiento de disco consiste en coger el disco superior de una de las torres y colocarlo en la cima de otra de las torres. </li>
    <li> Un disco nunca podrá estar sobre otro disco de menor tamaño. </li>
</ul>

Podéis ver aquí un ejemplo animado de Hanoi con 4 discos en la que queremos mover todos los discos desde la torre izquierda a la derecha.
<center><img src="img/hanoi.gif"></center>

Para más ejemplos y experimentación, tenéis disponible el juego online: https://www.mathsisfun.com/games/towerofhanoi.html
    
Se os pide que implementéis la solución de Programación Dinámica al problema de las torres de Hanoi para calcular el número mínimo de movimientos necesarios para mover $D$ discos de la torre $t_o$ a la torre $t_d$.

Además, se os pide que devolváis una lista de tuplas con la forma $(t_i,t_j)$ que simboliza mover el disco que se encuentra más arriba de la torre $i$ a la cima de la torre $j$.
Por ejemplo, para 3 discos, los movimientos serían:
[(0, 2), (0, 1), (2, 1), (0, 2), (1, 0), (1, 2), (0, 2)]
</p>

In [14]:
import numpy as np
def exclude(a,b):
    return 3-(a+b)

def dynamic_hanoi(D,t0=0,t1=2):
    #Create matrix
    num_moves= np.zeros((D,3,3),dtype=np.integer)
    movements= dict()
    
    for i in range(3):
        for j in range(3):
            num_moves[0,i,j] = 1 if i!=j else 0
            
    for d in range(1,D):
        for i in range(3):
            for j in range(3):
                if i!=j:
                    eTower = exclude(i,j)
                    num_moves[d,i,j] = num_moves[d-1,i,eTower]+num_moves[0,i,j] + num_moves[d-1,eTower,j]
                    movements[(d,i,j)] = [(d-1,i,eTower),(0,i,j),(d-1,eTower,j)]
        
    def get_move_ordering(move):
        moves =[]
        if move[0] == 1:
            return [(m[1],m[2]) for m in movements[move]]
        else:
            for i in range(3):
                next_move= movements[move][i]
                if next_move[0] == 0:
                    moves.append((next_move[-2],next_move[-1]))
                else:
                    for n in get_move_ordering(next_move):
                        moves.append(n)
            return moves

    current_move = (D-1,t0,t1)
    num_moves= num_moves[current_move]
    moves = get_move_ordering(current_move)
    
    return num_moves ,moves
                                    

In [15]:
# Prueba tu algoritmo
# Mover n discos debería resultar en (2^n -1) movimientos

print(dynamic_hanoi(3))
print(dynamic_hanoi(4))
print(dynamic_hanoi(5))
print(dynamic_hanoi(6))
print(dynamic_hanoi(7))

(7, [(0, 2), (0, 1), (2, 1), (0, 2), (1, 0), (1, 2), (0, 2)])
(15, [(0, 1), (0, 2), (1, 2), (0, 1), (2, 0), (2, 1), (0, 1), (0, 2), (1, 2), (1, 0), (2, 0), (1, 2), (0, 1), (0, 2), (1, 2)])
(31, [(0, 2), (0, 1), (2, 1), (0, 2), (1, 0), (1, 2), (0, 2), (0, 1), (2, 1), (2, 0), (1, 0), (2, 1), (0, 2), (0, 1), (2, 1), (0, 2), (1, 0), (1, 2), (0, 2), (1, 0), (2, 1), (2, 0), (1, 0), (1, 2), (0, 2), (0, 1), (2, 1), (0, 2), (1, 0), (1, 2), (0, 2)])
(63, [(0, 1), (0, 2), (1, 2), (0, 1), (2, 0), (2, 1), (0, 1), (0, 2), (1, 2), (1, 0), (2, 0), (1, 2), (0, 1), (0, 2), (1, 2), (0, 1), (2, 0), (2, 1), (0, 1), (2, 0), (1, 2), (1, 0), (2, 0), (2, 1), (0, 1), (0, 2), (1, 2), (0, 1), (2, 0), (2, 1), (0, 1), (0, 2), (1, 2), (1, 0), (2, 0), (1, 2), (0, 1), (0, 2), (1, 2), (1, 0), (2, 0), (2, 1), (0, 1), (2, 0), (1, 2), (1, 0), (2, 0), (1, 2), (0, 1), (0, 2), (1, 2), (0, 1), (2, 0), (2, 1), (0, 1), (0, 2), (1, 2), (1, 0), (2, 0), (1, 2), (0, 1), (0, 2), (1, 2)])
(127, [(0, 2), (0, 1), (2, 1), (0, 2), (1, 0)

<div class="alert alert-success" style="width:90%; margin:0 auto;">

  <h2><p>3 - Algoritmo Floyd-Warshall</p></h2>
  
  <p> El algoritmo Floyd-Warshall es un algoritmo de programación dinámica que se emplea para encontrar los caminos mínimos en un grafo con pesos (que no tenga ciclos negativos) entre todos los pares de nodos. Se basa en ir construyendo una matriz con caminos intermedios entre trios de nodos. Podéis ver más información y consultar el pseudocódigo <a href="https://es.wikipedia.org/wiki/Algoritmo_de_Floyd-Warshall">aquí</a>.</p>

In [ ]:
''' 
Implementa aquí el algoritmo Floyd Warshall
'''

def print_mat(m):
    for r in m:
        if type(r[0]) is float or type(r[1]) is float:
            print([round(x,1) for x in r])
        else:
            print(r)
def floyd_warshall(G, orig, dest):
    N = len(G)
    D = [[float("inf") for _ in range(N)] for _ in range(N)]
    P = [[None for _ in range(N)] for _ in range(N)]
    for v in G:
        D[v][v] = 0
        for u in G[v]:
            D[u][v] = G[u][v]['weight']
            P[u][v] = None
    
    for k in range(N):
        for i in range(N):
            for j in range(N):
                d = D[i][k] + D[k][j]
                if D[i][j] >d:
                    D[i][j] = d
                    P[i][j] = k
        
    
    print_mat(P)
    path =[dest]
    while path[-1] != None:
        path.append(P[path[-1]][orig])
        
    

In [24]:
# defining the number of vertices
nV = 4
INF = 999
 
def floydWarshall(graph):
    dist = list(map(lambda i : list(map(lambda j : j, i)), graph) )
    
    for k in range(nV): 
        for i in range(nV): 
            for j in range(nV): 
                dist[i][j] = min(dist[i][j], dist[i][k]+ dist[k][j]) 
    printSolution(dist) 
def printSolution(dist): 
    for i in range(nV): 
        for j in range(nV): 
            if(dist[i][j] == INF): 
                print("INF", end =" ")
            else: 
                print(dist[i][j], end ="  ")  
        print(" ")
graph = [[0, 3, INF, 5],
         [2, 0, INF, 4],
         [INF, 1, 0, INF],
         [INF, INF, 2, 0]]; 
floydWarshall(graph);

0  3  7  5   
2  0  6  4   
3  1  0  5   
5  3  2  0   


In [3]:
from math import inf
from itertools import product
def floyd_warshall(n, edge):
    rn = range(n)
    dist = [[inf] * n for i in rn]
    nxt = [[0] * n for i in rn]
    for i in rn:
        dist[i][i] = 0
    for u, v, w in edge:
        dist[u - 1][v - 1] = w
        nxt[u - 1][v - 1] = v - 1
    for k, i, j in product(rn, repeat=3):
        sum_ik_kj = dist[i][k] + dist[k][j]
        if dist[i][j] > sum_ik_kj:
            dist[i][j] = sum_ik_kj
            nxt[i][j] = nxt[i][k]
    print("    pair        dist     path")
    for i, j in product(rn, repeat=2):
        if i != j:
            path = [i]
            while path[-1] != j:
                path.append(nxt[path[-1]][j])
            print("%3d → %3d  %4d       %s"
                  % (i + 1, j + 1, dist[i][j],
                     ' → '.join(str(p + 1) for p in path)))
if __name__ == '__main__':
    floyd_warshall(4, [[1, 3, -2], [2, 1, 4], [2, 3, 3], [3, 4, 2], [4, 2, -1]])

    pair        dist     path
  1 →   2    -1       1 → 3 → 4 → 2
  1 →   3    -2       1 → 3
  1 →   4     0       1 → 3 → 4
  2 →   1     4       2 → 1
  2 →   3     2       2 → 1 → 3
  2 →   4     4       2 → 1 → 3 → 4
  3 →   1     5       3 → 4 → 2 → 1
  3 →   2     1       3 → 4 → 2
  3 →   4     2       3 → 4
  4 →   1     3       4 → 2 → 1
  4 →   2    -1       4 → 2
  4 →   3     1       4 → 2 → 1 → 3


<div class="alert alert-success" style="width:90%; margin:0 auto;">
  <h2><p>3.5 - Johnson</p></h2>
  

In [5]:
def APSP_Johnson(graph):
    """
    Applies Johnson's to compute all-pairs
    shortest paths.
    """
    n = graph.numVertices()
    for v in graph.vertices():
        graph.addDirectedEdge(n, v, 0)
        
    h, _ = BellmanFord(graph, n)

    del graph.adjacent[n]
    for v in graph.vertices():
        del graph.weight[(n, v)]

    if h == None:
        return None

    for (u, v) in graph.edges():
        graph.weight[(u, v)] += h[u] - h[v]

    dst = [None for u in range(n)]
    nxt = [None for u in range(n)]

    for u in graph.vertices():
        dst[u], nxt[u] = Dijkstra(graph, u)

    for u in graph.vertices():
        for v in graph.vertices():
            delta_h = h[u] - h[v]
            dst[u][v] -= delta_h
            if (u, v) in graph.weight:
                graph.weight[(u,v)] -= delta_h

    return (dst, nxt)


def Dijkstra(graph, s):
    """
    Applies Dijkstra to compute single-source
    shortest paths.
    """
    dist = [float('inf') for v in graph.vertices()]
    parent = [None for v in graph.vertices()]

    queue = Min_Heap()
    queue.insert(0, s)

    while queue:
        d, u = queue.extract_min()
        dist[u] = d

        for v in graph.adjacent[u]:
            new_dist = d + graph.weight[(u,v)]

            if v not in queue:
                if new_dist < dist[v]:
                    queue.insert(new_dist, v)
                    parent[v] = u

            elif new_dist < queue.key(v):
                queue.decrease_key(v, new_dist)
                parent[v] = u

    return (dist, parent)


def BellmanFord(graph, s):
    """
    Applies Bellman-Ford to compute single-source
    shortest paths.
    """
    dst = [float('inf') for v in graph.vertices()]
    pnt = [None for V in graph.vertices()]

    dst[s] = 0
    numRelaxations = graph.numVertices()-1
    for relaxation in range(numRelaxations):
        for (u, v) in graph.edges():
            new_dst = dst[u] + graph.weight[(u,v)]
            if new_dst < dst[v]:
                dst[v] = new_dst
                pnt[v] = u 

    if any(dst[u] + graph.weight[(u,v)] < dst[v] for (u, v) in graph.edges()):
        return (None, None)
    
    return (dst, pnt)


def showResults(graph, dst, pointer):
    """
    Shows all-pairs shortest paths information.
    """
    if dst == None:
        print(None)
        return

    print("Distances:")
    for (v, row) in zip(graph.vertices(), dst):
        print(f"{v}: {row}")

    print("\nPath Pointers:")
    for (v, row) in zip(graph.vertices(), pointer):
        print(f"{v}: {row}")

if __name__ == '__main__':
    from graph import Graph
    from heap import Min_Heap

    graph = Graph()
    print(f"\nGraph:\n{graph}")

    print("\n\nDynamic Programming:")
    print("-"*25)
    dst, nxt = APSP_DynamicProgramming(graph)
    showResults(graph, dst, nxt)
    
    print("\n\nFloyd - Warshall")
    print("-"*25)
    dst, nxt = APSP_FloydWarshall(graph)
    showResults(graph, dst, nxt)

    print("\n\nJohnson")
    print("-"*25)
    dst, prev = APSP_Johnson(graph)
    showResults(graph, dst, prev)
    print()

ModuleNotFoundError: No module named 'graph'

<div class="alert alert-success" style="width:90%; margin:0 auto;">
  <h2><p>4-DTW</p></h2>
  
  <div class="row">
  <div>
  <p style="text-align: justify; padding-right: 30px">
  Implementa el algoritmo DTW presentado en clase de teoría. Para ello, las entradas serán 2 vectores, el patron a buscar, y el vector donde encontrar el patrón. Usaremos valores reales en cada posición del vector, que hipotéticamente representarán valores de un sensor inercial situado en la muñeca de un usuario que realiza gestos con el objetivo de interaccionar con una interfaz. El código debe implementar DTW de tal forma que la salida sea, el coste de la asociación más factible, mostrando la posición de inicio-fin dentro del segundo vector.
  </p>
  </div>

In [18]:
def DTW(V1,V2):
    #The shortest word will always be V1
    if len(V1)>len(V2):V1,V2=V2,V1
    #We make a matrix 1 row and 1 column greater than what we need to avoid problems
    l1,l2=len(V1)+1,len(V2)+1
    #Our matrix will store the current distance in the first position and its origin in the second, in where:
    #    0=diagonal
    #    1=up
    #    2=left
    m=[[(0,0) for i in range(l2)]for i in range(l1)]
    
    #First column will set to infinite, because even if the algorithm doesn't care about time
    #It cares about the pattern
    for i in m[1:]:
        i[0]=(float('inf'),0)

    #Fill the matrix
    for i in range(1,l1):
        for j in range(1,l2):
            ins=m[i][j-1][0]+1*(V2[i-1]!=V2[i-2])
            sub=m[i-1][j-1][0]+1*(V1[i-1]!=V2[j-1])
            el=m[i-1][j][0]+1
            mm=[sub,el,ins]
            minm=min(mm)
            m[i][j]=(minm,mm.index(minm))
    
    for i in m:
        print (i)
    
    last=min(m[-1][1:], key=lambda x:x[0])
    coste=last[0]
    fin=m[-1][1:].index(last)
    
    i=l1-1
    inicio=fin+1
    while i!=0:
        if m[i][inicio][1]==0:
            i-=1
            inicio-=1
        elif m[i][inicio][1]==1:
            i-=1
        else:
            inicio-=1

    return (coste, inicio, fin)

DTW('asdf' , 'ashf')

[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
[(inf, 0), (0, 0), (1, 0), (1, 0), (1, 0)]
[(inf, 0), (1, 1), (0, 0), (1, 2), (2, 0)]
[(inf, 0), (2, 1), (1, 1), (1, 0), (2, 0)]
[(inf, 0), (3, 1), (2, 1), (2, 0), (1, 0)]


(1, 0, 3)

In [17]:
def DTW_R(V1,V2):
    #The shortest word will always be V1
    if len(V1)>len(V2):V1,V2=V2,V1
    #We make a matrix 1 row and 1 column greater than what we need to avoid problems
    l1,l2=len(V1)+1,len(V2)+1
    #Our matrix will store the current distance in the first position and its origin in the second, in where:
    #    0=diagonal
    #    1=up
    #    2=left
    m=[[(0,0) for i in range(l2)]for i in range(l1)]
    
    #First column will set to infinite, because even if the algorithm doesn't care about time
    #It cares about the pattern
    for i in range(1,l1):
        m[i][0]=(float('inf'),0)

    #Fill the matrix
    for i in range(1,l1):
        for j in range(1,l2):
            mm=[m[i-1][j-1][0],m[i-1][j][0],m[i][j-1][0]]
            minm=min(mm)
            m[i][j]=(minm+abs(V1[i-1]-V2[j-1]),mm.index(minm))
    
    last=min(m[-1][1:], key=lambda x:x[0])
    coste=last[0]
    fin=m[-1][1:].index(last)
    
    i=l1-1
    inicio=fin+1
    while i!=0:
        if m[i][inicio][1]==0:
            i-=1
            inicio-=1
        elif m[i][inicio][1]==1:
            i-=1
        else:
            inicio-=1

    return (coste, inicio, fin)

DTW_R([0.1,0.3,1.2,1.7],[0.2,0.4,0.7,1.1,1.6])

(0.7999999999999997, 0, 4)


<div class="alert alert-success" style="width:90%; margin:0 auto;">
  <h2><p>5 - Problema de la mochila</p></h2>

In [10]:
def knapSack(W, wt, val,n ):
    K = [[0 for x in range(W+1)] for x in range(n+1)]
    
    for i in range(n+1):
        for w in range(W+1):
            if i == 0 or w ==0:
                K[i][w] =0
            elif wt[i-1] <=w:
                K[i][w] = max(val[i-1] + K[i-1] [w-wt[i-1]], K[i-1][w])
            else:
                K[i][w] = K[i-1][w]
    return K[n][w]
val = [60,100,120]
wt = [10,20,30]
W = 50
n = len(val)
print(knapSack(W,wt,val,n))

220


<div class="alert alert-success" style="width:90%; margin:0 auto;">
  <h2><p>6 - TSP(Dynamic program)</p></h2>

In [6]:
def solve_tsp_dynamic(points):
    #calc all lengths
    all_distances = [[length(x,y) for y in points] for x in points]
    #initial value - just distance from 0 to every other point + keep the track of edges
    A = {(frozenset([0, idx+1]), idx+1): (dist, [0,idx+1]) for idx,dist in enumerate(all_distances[0][1:])}
    cnt = len(points)
    for m in range(2, cnt):
        B = {}
        for S in [frozenset(C) | {0} for C in itertools.combinations(range(1, cnt), m)]:
            for j in S - {0}:
                B[(S, j)] = min( [(A[(S-{j},k)][0] + all_distances[k][j], A[(S-{j},k)][1] + [j]) for k in S if k != 0 and k!=j])  #this will use 0th index of tuple for ordering, the same as if key=itemgetter(0) used
        A = B
    res = min([(A[d][0] + all_distances[0][d[1]], A[d][1]) for d in iter(A)])
    return res[1]

<div class="alert alert-info">
<center>
  <h1>Enumerativos</h1>
</center>
</div>


<div class="alert alert-success" style="width:90%; margin:0 auto;">
  <h2><p>1. Problema de la mochila</p></h2>
  
   <p> En esta primera sección trabajaremos con el problema de la mochila que ya vimos en la práctica de Greedy y problemas de PD. Refrequemos un poco (por si aún no lo tenéis claro):
</p>
    Nos encontramos en una habitación en la que hay $N$ objetos con pesos $w_1, w_2, w_3 ... w_N$ y tenemos una mochila que puede soportar una carga máxima de $W$. En este caso se pide que realicéis una implementación de Ramificación y Poda para resolver el problema de la mochila con valor:
<br><br>
 **mochila_valor:** Cada objeto tendrá asignado un valor $v_1, v_2, v_3 ... v_N$. Buscamos llenar la mochila maximizando el valor total de la mochila pero sin superar la capacidad máxima. Es decir queremos encontrar la combinación de objetos $b$ tal que $\arg_{b} \max{\sum_{i=0}^{N}{v_i · b_i}}$ con la condición de que $\sum_{i=0}^{N}{(w_i · b_i)}\leq W$.
     <br><br>
Trabajaremos únicamente con la versión de la mochila en la que tenemos una cantidad limitada $c_1, c_2, c_3 ... c_N$ de objetos.</div>

In [1]:
import random as rd

# Genera un conjunto de objetos de prueba en forma de una lista de listas, 
# cada una de las cuales representa un elemento [peso,valor,cantidad]
def random_objects():
    objects = [[p,v,c] for p,v,c in zip(rd.sample(range(1, 50), k=10), rd.sample(range(1, 100), k=10),rd.sample(range(1, 15), k=10))]
    return objects

In [2]:
import heapq
import itertools
''' 
Implementa aquí la solución de Ramificación y poda que resuelve el problema de la mochila
@input: Lista de listas con la forma [peso,valor,cantidad] representando los objetos que podemos escoger.
@output: Lista de listas con la forma [peso,valor,cantidad] representando los objetos escogidos.
'''
def knapsack_branch(D, W):
    global best
    best = [0, []]
    D = sorted(D,key=lambda x: x[1]/x[0], reverse=True)
    knapsack_branch_re(D, W, (0, 0, [0 for i in range(len(D))]))
    print(best[0])
    return [(D[i][0], D[i][1], best[1][i]) for i in range(len(D)) if best[1][i] != 0]
            
def knapsack_branch_re(D, W, partial):
    global best
    end = True
    for i in range(len(D)):
        if 0 < D[i][2] - partial[2][i] and partial[0] + D[i][0] <= W:
            end = False
            tmp = partial[2].copy()
            tmp[i] +=1
            partialCopy = (partial[0] + D[i][0], partial[1] + D[i][1], tmp)
            if isNotPodable(D, W, partialCopy):
                knapsack_branch_re(D, W, partialCopy)
    if end and best[0] < partial[1]:
        best[0] = partial[1]
        best[1] = partial[2]
        print(partial)

def isNotPodable(D, W, partial):
    global best
    j=0
    peso = partial[0]
    valor = partial[1]
    while j < len(D):
        if D[j][2] - partial[2][j] > 0 and peso + D[j][0] <= W:
            n = min((W - peso) // D[j][0], D[j][2] - partial[2][j])
            peso += n * D[j][0]
            valor += n * D[j][1]
        else:
            j = len(D)
        j += 1
    n = (W - peso) / D[-1][0]
    peso += round(n * D[-1][0])
    valor += round(n * D[-1][1])
    return best[0] < valor 

In [3]:
import random
W = 150
for i in range(10):
    random.seed(i)
    D = random_objects()
    print(knapsack_branch(D,W))

(150, 1130, [10, 6, 0, 0, 0, 0, 0, 0, 0, 0])
1130
[(3, 65, 10), (20, 80, 6)]
(150, 1010, [10, 5, 0, 0, 0, 0, 0, 0, 0, 0])
1010
[(8, 56, 10), (14, 90, 5)]
(148, 1017, [6, 7, 1, 0, 0, 0, 0, 0, 0, 1])
(146, 1018, [6, 7, 0, 1, 0, 0, 0, 0, 0, 0])
(148, 1028, [6, 6, 2, 0, 0, 0, 0, 0, 0, 0])
1028
[(4, 78, 6), (14, 66, 6), (20, 82, 2)]
(145, 910, [8, 6, 1, 0, 0, 0, 0, 0, 0, 0])
910
[(5, 51, 8), (16, 78, 6), (9, 34, 1)]
(148, 1786, [7, 5, 9, 9, 0, 0, 0, 0, 0, 0])


KeyboardInterrupt: 

<div class="alert alert-success" style="width:90%; margin:0 auto;">

  <h2><p>2 - Saltos de caballo</p></h2>
  
  <p>
 En el ajedrez, un caballo se mueve en forma de "L" en cualquier dirección posible. Es decir, se mueve en línea recta 2 posiciones y luego 1 más a izquierda o derecha. Todos los movimientos posibles de un caballo en una posición dada puden verse en la siguiente imagen:

<p><center><img src='img/horse_moves.jpg' width=30%></img></center></p>

<p>El examen consta de un único ejercicio en el cual tendréis que implementar un algoritmo de ramificación y poda (recursivo o iterativo) que, dado un tablero de tamaño $n \times n$ y un número de casillas $C$, devuelva el recorrido de caballo más corto que pasa por $C$ casillas diferentes (contando la inicial, la cual es el tercer parámetro de la función).
</p>

Os damos ya implementada la clase HorseState. Podéis modificar la clase como considréis necesario o podéis incluir otras funciones de interés para la resolución del problema. Si empleáis alguna variable global recordad que es necesario declararlas dentro de las funciones como <i>global</i> en caso de que queramos modificarlas. Por ejemplo:

```
global_variable = 0

def function_that_reads_global_var(param):
    if global_variable > param:
        print("It is greater!")

def function_that_modifies_global_var(param):
    global global_variable # OJO!
    if global_variable > param:
        global_variable = param
```

Por último, también os pedimos que retornéis la cantidad de estados que ha sido necesario explorar hasta encontrar la solución óptima.


In [13]:
# La función deepcopy(x) retorna una copia del objeto x 
# de manera que lo podemos modificar sin que afecte al original
from copy import deepcopy

# Os damos la clase HorseState como base para que realicéis vuestro trabajo
class HorseState:
    def __init__(self,N,S,C,sol=None,value=0, unique = 1):
        self.size = N # Tamaño del tablero
        self.objective = C # Número de celdas distintas a visitar
        self.sol = sol if sol else [S]  # Camino construido hasta ahora, la posición actual del caballo es self.sol[-1]
        self.value = value + 1 # Peso total de la solución (equivalente a len(self.sol))
        self.unique = unique #Numero de elementos no reptidos
    
    # Devuelve las posiciones accesibles para un caballo desde la celda actual (la última añadida a la solución).
    def possible_next(self):
        (x,y) = self.sol[-1]
        possible = []
        if x < self.size and y < self.size:
            if x-2 >= 0:
                if y-1 >= 0:
                    possible.append((x-2,y-1))
                if y+1 < self.size:
                    possible.append((x-2,y+1))
            if x+2 < self.size:
                if y-1 >= 0:
                    possible.append((x+2,y-1))
                if y+1 < self.size:
                    possible.append((x+2,y+1))
            if y-2 >= 0:
                if x-1 >= 0:
                    possible.append((x-1,y-2))
                if x+1 < self.size:
                    possible.append((x+1,y-2))
            if y+2 < self.size:
                if x-1 >= 0:
                    possible.append((x-1,y+2))
                if x+1 < self.size:
                    possible.append((x+1,y+2))
        return possible
    
    # Devuelve un nuevo estado añadiendo la celda
    def add_cell(self,cell):
        return HorseState(self.size,self.sol[0],self.objective,self.sol+[cell],self.value,self.unique+(1 if cell not in self.sol else 0))
    
    # LessThan compara el objeto actual con otro 
    # (lo utilizan clases como PriorityQueue o HeapQ para ordenar los objetos)
    def __lt__(self,other):
        return self.unique > other.unique
    
    # Convierte el objeto a string (para formatear los prints)
    def __str__(self):
        return str(self.sol) + " " + str(self.value) + " " + str(self.unique)
    
    #Comprueba si puede ser un camino mejor
    def isPosibleBetter(self,best):
        return self.value - self.unique  < best.value - best.unique #== 0
            

In [14]:
import heapq
'''
Implementad aquí vuestra solución:
@input:     N --> tamaño del tablero.
            S --> posición inicial del caballo en forma de tupla.
            C --> número de celdas diferentes a recorrer (como máximo N²).
@output:    path --> Lista de celdas que representan el recorrido mínimo pasando por C celdas diferentes.
            expanded --> Número de nodos explorados antes de encontrar la solución óptima.
''' 
def min_horse_path(N,S,C):
    el = HorseState(N,S,C)
    heap = [el]
    expanded = 0
    best = HorseState(N,S,C,value=float("inf"))
    while len(heap) and best.unique!=C:
        el = heapq.heappop(heap)
        if el.value == C and el.unique > best.unique:
            best = el
        else:
            for i in el.possible_next():
                tmp = el.add_cell(i)
                if tmp.isPosibleBetter(best):
                    heapq.heappush(heap,tmp)
        expanded+=1
    return best.sol,expanded

In [15]:
path,_ = min_horse_path(5,(0,0),25)
assert len(path) == 25

<div class="alert alert-warning" style="width:80%; margin:0 auto; padding">
<center><p><h3> Comentarios y análisis de complejidad</h3></p> </center> </div>
<b> Explica brevemente la estrategia que has empleado para resolver el problema, enfatizando los mecanismos que reducen el espacio de búsqueda, y realiza el análisis de complejidad de tu algoritmo. </b>


El algoritmo prioriza encontra una solucion con el maximo numero de casillas unicas (no repetidas) solo podara los caminos que tengan mayor o igual numero de casillas repetidas (se puede optimizar un poco suponinendo que buscamos simpre una solucion perfecta pero no estoy seguro que siempre se pueda encontrar sin repetir casillas. Finalmente el algoritmo finaliza si encuentra que ha alcanzado una solucion optima, basicamente que no haya ninguna casilla repetida en la mejor solucion.


<div class="alert alert-success" style="width:90%; margin:0 auto;">
  <h2><p>3 - Problema de las 8 reinas</p></h2>
  

In [ ]:
"""The n queens puzzle"""
class NQueens:
    """Generem totes les posible solucions al problema"""
    def __init__(self,size):
        # Store the puzzle (problem) size and the number of valid solutions
        self.size = size
        self.solutions = 0
        self.solve()
        
    def solve(self):
        """Resol el puzzle de les n reines i imprimeix el numero de solucions"""
        positions = [-1] *self.size
        self.put_queen(positions,0)
        print("Trobades", self.solutions, "solucions.")
    
    def put_queen(self,positions, target_row):
        """
        Intentem posar una reina a un target_row comprovant els n posibles cassos.
        Si la posicio trobada es valida la funcio es cridara a si mateixa per intentar posar la reina
        a la seguent columna fins que les n reines estiguin posades al NxN board
        """
        #El cas base para si totes les n columnes estan ocupades
        if target_row == self.size:
            self.show_full_board(positions)
            self.solutions +=1
        else:
            #Per totes les n columnes intentar posar la reina
            for column in range(self.size):
                #No agafem les posicions invalides
                if self.check_place(positions, target_row, column):
                    positions[target_row] = column
                    self.put_queen(positions, target_row+1)
    
    def check_place(self, positions, ocuppied_rows, column):
        """
        Comprovem si alguna de les posicions pot estar sota atack
        des d'alguna posicio de les altres reines(Comprovem les columnes
        i posicions diagonals)
        """
        for i in range(ocuppied_rows):
            if positions[i] == column or \
            positions[i]-1 == column - ocuppied_rows or  \
            positions[i]+1 == column + ocuppied_rows:
                return False
        return True
    
    def show_full_board(self, positions):
        """Show the full NxN board"""
        for row in range(self.size):
            line = ""
            for column in range(self.size):
                if positions[row] == column:
                    line += "Q "
                else:
                    line += ". "
            print(line)
        print("\n")

def main():
    """Iniialize and solve the n queens puzzle"""
    NQueens(8)
if __name__ == "__main__":
    #execute only if run as script
    main()


<div class="alert alert-success" style="width:90%; margin:0 auto;">
  <h2><p> 4 - Problema de la rata en el laberinto(BackTracking)</p></h2>

In [26]:
SIZE = 5
#maze problem
maze = [
    [0,1,0,1,1],
    [0,0,0,0,0],
    [1,0,1,0,1],
    [0,0,1,0,0],
    [1,0,0,1,0]
]
#list to store the solution matrix
solution = [[0]*SIZE for _ in range(SIZE)]

#function to solve the maze
#using backtracking
def solvemaze(r, c):
    #if destination is reached, maze is solved
    #destination is the last cell(maze[SIZE-1][SIZE-1])
    if (r==SIZE-1) and (c==SIZE-1):
        solution[r][c] = 1;
        return True;
    #checking if we can visit in this cell or not
    #the indices of the cell must be in (0,SIZE-1)
    #and solution[r][c] == 0 is making sure that the cell is not already visited
    #maze[r][c] == 0 is making sure that the cell is not blocked
    if r>=0 and c>=0 and r<SIZE and c<SIZE and solution[r][c] == 0 and maze[r][c] == 0:
        #if safe to visit then visit the cell
        solution[r][c] = 1
        #going down
        if solvemaze(r+1, c):
            return True
        #going right
        if solvemaze(r, c+1):
            return True
        #going up
        if solvemaze(r-1, c):
            return True
        #going left
        if solvemaze(r, c-1):
            return True
        #backtracking
        solution[r][c] = 0;
        return False;
    return 0;
if(solvemaze(0,0)):
    for i in solution:
        print (i)
else:
    print ("No solution")

[1, 0, 0, 0, 0]
[1, 1, 1, 1, 0]
[0, 0, 0, 1, 0]
[0, 0, 0, 1, 1]
[0, 0, 0, 0, 1]


In [ ]:
"""
PROBLEMA DE LA RATA V2
"""

# Utility to check if the current cell position (x,y)
# is in the maze
def isSafe(maze, x, y, sol):
    # Get maze dimensions
    X = len(maze[1])
    Y = len(maze)

    if x>=0 and x<X and y>=0 and y<Y and maze[x][y]==1:    
        return True
    return False

# (x,y) is the current cell position
def solveMaze(maze, x, y, sol):

    # Get maze size
    X = len(maze[1])
    Y = len(maze)

    # check if (x,y) is goal
    if x == X-1 and y == Y-1 : 
        sol[x][y] = 1
        return True

    # Check if we're inside the maze
    if isSafe(maze, x, y, sol):
        # Mark the current cell in solution (BACKTRACK)
        sol[x][y] = 1
        # Move right
        if solveMaze(maze, x+1, y, sol):
            return True
        # Move down
        if solveMaze(maze, x, y+1, sol):
            return True
        # Remove current cell from solution
        # If the 2 moves above failed
        sol[x][y] = 0
        return False

maze = [
    [1, 0, 0, 0],
    [1, 1, 0, 1],
    [0, 1, 0, 0],
    [1, 1, 1, 1,]
]
# Initiate the solution matrix
sol = [
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
]

# Given a maze NxM,
# we start at (0, 0), goal is (N-1, M-1)
if solveMaze(maze, 0, 0, sol):
    print sol
else:
    print "No solution"


<div class="alert alert-success" style="width:90%; margin:0 auto;">
  <h2><p> 5 - Problema de los colores(BackTracking)</p></h2>

In [29]:
def is_safe(n, graph, colors, c):
    # Iterate trough adjacent vertices
    # and check if the vertex color is different from c
    for i in xrange(n):
        if graph[n][i] and c == colors[i]: return False
    return True

# n = vertex nb
def graphColoringUtil(graph, color_nb, colors, n):
    # Check if all vertices are assigned a color
    if color_nb+1 == n :
        return True

    # Trying differents color for the vertex n
    for c in xrange(1, color_nb+1):
        # Check if assignment of color c to n is possible
        if is_safe(n, graph, colors, c):
            # Assign color c to n
            colors[n] = c
            # Recursively assign colors to the rest of the vertices
            if graphColoringUtil(graph, color_nb, colors, n+1): return True
            # If there is no solution, remove color (BACKTRACK)
            colors[n] = 0
            
#nb of vertex
vertex_nb = 4
# nb of colors
color_nb = 3
# Initiate vertex colors
colors = [0] * vertex_nb

graph = [
    [0, 1, 1, 1],
    [1, 0, 1, 0],
    [1, 1, 0, 1],
    [1, 0, 1, 0],
]

#beginning with vertex 0
if graphColoringUtil(graph, color_nb, colors, 0):
    print (colors)
else:
    print ("No solutions")

NameError: name 'xrange' is not defined


<div class="alert alert-success" style="width:90%; margin:0 auto;">
  <h2><p>Ramificación y poda: 6 - Problema de la mochila</p></h2>
  
  <div class="row">
  <div>
  <p style="text-align: justify; padding-right: 30px">
  Implementa mediante ramificación y poda una solución al problema de la mochila con tal de encontrar el listado ordenado de items a introducir en la mochila de tal forma que el beneficio (en términos de valor) acumulado en la mochila sea máximo.
  </p>
  </div>


<div class="alert alert-danger" style="width:80%; margin:0 auto; padding">
<center><p><h3> Código </h3></p> </center> 

<p>
<h3>INPUT</h3>
<ul>
<li>V1: vector del peso asociado a cada ítem (cada posición corresponde a cada uno de los n ítems).</li>
<li>V2: vector del valor asociado a cada ítem (cada posición corresponde a cada uno de los n ítems).</li>
<li>V3: vector con la cantidad de elementos disponibles de cada tipo de ítem (cada posición corresponde a cada uno de los n ítems).</li>
<li>X: peso máximo que soporta la mochila a ser rellenada por ítems con el objetivo de maximizar el valor que contiene.</li>
</ul>
<br>
<h3>OUTPUT</h3>
Listado de elementos que se introducen en la mochila (en el orden en el que se introducen, índice del ítem, la longitud del vector es la cantidad de elementos que se introducen en la mochila) y el valor total de la mochila.
</p>

</div>

In [19]:
import copy
def RP(V1,V2,V3,X):
    d=dict([(i,{'weight':V1[i],'value':V2[i],'amount':V3[i]}) for i in range(len(V1))])
    items=[]
    valor=0
    peso=0
    expanded=[]
    temp=[(items,valor,peso,d)]
    
    #First, we have to find a minimal value
    while temp:
        items,valor,peso,d=temp.pop()
        #To avoid permutations, we sort our list in order to have all permutations as one
        items.sort()
        
        #If weight is already greater than what the knapsack can handle, we have an initial solution
        if peso>=X:
            valor_max=valor
            break
        else:
            #Having a list of lists doesn't work if we want to check if a list is in it
            #So we convert it into a string
            s_items=list_to_string(items)
            if s_items not in expanded:
                expanded.append(s_items)
                
                #Get the elements we can pick (those that are left) and sort them by its value
                case=([(i,d[i]['value']) for i in d if d[i]['amount']!=0])
                case.sort()
                
                #For every item, we check if the weight is already greater than X and
                #if it is, we stop
                #if not, we add it to our list
                for i,v in case:
                    if peso+d[i]['weight']<=X:
                        #To avoid modifying the original list for the items that are yet to come
                        #we copy the dictionary and then modify its values
                        temp_d=copy.deepcopy(d)
                        temp_d[i]['amount']-=1
                        temp.append((items+[i],valor+v,peso+d[i]['weight'],temp_d))
                    else:
                        valor_max=valor
                        break
                        
    #From here, we would have to use our max_valor to bound those nodes that can't give us a better solution
    #But I haven't been able to find a bound condition, so it returns the solution found in the DFS
    return items,valor_max

def list_to_string(l):
    s=''
    for i in l:
        s+=str(i)
    return s

def bound(tree):
    pass

RP([1,2,3],[4,5,6],[11,2,7],10)

([0, 2, 2, 2], 22)


<div class="alert alert-success" style="width:90%; margin:0 auto;">
  <h2><p> 7 - Mochila</p></h2>
  
   <p> En esta primera sección trabajaremos con el problema de la mochila que ya vimos en problemas Greedy y PD. Refrequemos un poco (por si aún no lo tenéis claro):
</p>
    Nos encontramos en una habitación en la que hay $N$ objetos con pesos $w_1, w_2, w_3 ... w_N$ y tenemos una mochila que puede soportar una carga máxima de $W$. En este caso se pide que realicéis una implementación de Ramificación y Poda para resolver el problema de la mochila con valor:
<br><br>
 **mochila_valor:** Cada objeto tendrá asignado un valor $v_1, v_2, v_3 ... v_N$. Buscamos llenar la mochila maximizando el valor total de la mochila pero sin superar la capacidad máxima. Es decir queremos encontrar la combinación de objetos $b$ tal que $\arg_{b} \max{\sum_{i=0}^{N}{v_i · b_i}}$ con la condición de que $\sum_{i=0}^{N}{(w_i · b_i)}\leq W$.
     <br><br>
Trabajaremos únicamente con la versión de la mochila en la que tenemos una cantidad limitada $c_1, c_2, c_3 ... c_N$ de objetos.</div>


In [ ]:
import random as rd

# Genera un conjunto de objetos de prueba en forma de una lista de listas, 
# cada una de las cuales representa un elemento [peso,valor,cantidad]
def random_objects():
    objects = [[p,v,c] for p,v,c in zip(rd.sample(range(1, 50), k=10), rd.sample(range(1, 100), k=10),rd.sample(range(1, 15), k=10))]
    return objects

In [26]:
import copy
import heapq as hq

def greedy_knapsack(B):
    B.sort(key=lambda elem: elem[1]/elem[0],reverse=True)
    D = copy.deepcopy(B)
    x = []
    w = 0
    v = 0
    # Mientras no se cumpla el criterio de terminación
    while D and w + min(D,key=lambda elem: elem[0])[0] < W:
        # Escoge un elemento
        elem = D.pop(0)
        available_weight = W - w
        how_many = min([available_weight//elem[0],elem[2]])
        if how_many > 0:
            x.append([elem[0],elem[1],how_many])
            w += elem[0]*how_many
            v += elem[1]*how_many
    # Devuelve la solución y la métrica
    return x,w,v,D
''' 
Implementa aquí la solución de Ramificación y poda que resuelve el problema de la mochila
@input: (D) Lista de listas con la forma [peso,valor,cantidad] representando los objetos que podemos escoger.
        (W) Entero que representa la capacidad de la mochila.
@output: Lista de listas con la forma [peso,valor,cantidad] representando los objetos escogidos.
'''
class Knapsack:
    def __init__(self,D,W,x = None,w = 0, v = 0, b = 0):
        self.x = x if x else list()
        self.D = D
        self.W = W
        self.weight = w
        self.value = v
        self.best = D[0] if len(D) else b
    
    # Current value + expected better than best solution
    def is_promising(self,val):
        return (self.value + self.optimistic()) > val
    
    # The lightest object fits the remaining space
    def is_complete(self):
        return self.weight + min(self.D,key=lambda elem: elem[0])[0] >= self.W
    
    def optimistic(self,opt=1):
        if opt:
            # Assume we can fill knapsack with unlimited copies of the object with best weight/value ratio
            return ((self.W-self.weight)//self.best[0])*self.best[1]
        else:
            # Relax maximum knapsack capacity (grab every object remaining)
            return sum([obj[1]*obj[2] for obj in self.D])
    
    # Possible objects
    def possible(self):
        pos = []
        if not self.is_complete():
            available_space = self.W - self.weight
            for el,i in zip(self.D,range(len(self.D))):
                if el[0] <= available_space: 
                    pos.append((el,i))
            return pos
        else:
            return []
    
    # Add element to solution and remove from objects list
    def add_elem(self,elem):
        e,i = elem[0],elem[1]
        Daux = copy.deepcopy(self.D)
        Daux[i][2] -= 1
        if Daux[i][2] == 0:
            del Daux[i]
        
        return Knapsack(Daux,self.W,self.x+[[e[0],e[1],1]],self.weight + e[0],self.value + e[1],self.best)
    
    # Format the solution
    def compact(self):
        objects = dict() 
        for i in self.x:
            objects[(i[0],i[1])] = objects.get((i[0],i[1]),0) + i[2]
        return [[k[0],k[1],v] for k,v in objects.items()]
    
    # Compare two states
    def __lt__(self,other,opt=0):
        if opt:
            return self.optimistic() < other.optimistic()
        else:
            return (self.value + self.optimistic()) < (other.value + other.optimistic())
    
    def __str__(self):
        return "Remaining objects:" + str(self.D) + '\nSolution:' + str(sorted(self.compact())) + "; V " + str(self.value) + '; W ' + str(self.weight)
    
def knapsack_branch(D,W):
    # Initial solution
    b_x,b_w,b_v,extraD = greedy_knapsack(D)
    best = Knapsack(extraD,W,b_x,b_w,b_v)
    print('GREEDY SOLUTION FOUND: \n' + str(best))
    
    # Initialise exploration
    queue = [Knapsack(D,W)]
    expanded = 0
    explored_states = set()
    
    # Explore states
    while len(queue) > 0:
        # Get best (potencially) possible state
        current = hq.heappop(queue)
        if str(current) in explored_states: continue
        explored_states.add(str(current))
        expanded += 1
        # If can't add more objects and improves best solution
        if current.is_complete() and best.value < current.value:
            # Update best
            best = copy.deepcopy(current)
        # If current is worth exploring
        if current.is_promising(best.value):
            for el in current.possible():
                poss = current.add_elem(el)
                if str(poss) not in explored_states:
                    hq.heappush(queue,poss)
        
    return best


In [27]:
rd.seed(6) # fijando el seed obtenemos siempre la misma mochila
D = random_objects()
W = 150
print('AVAILABLE OBJECTS: \n' + str(D))
sol = knapsack_branch(D,W)
print('FINAL SOLUTION: \n' + str(sol))

AVAILABLE OBJECTS: 
[[37, 41, 11], [6, 99, 2], [32, 3, 4], [17, 35, 10], [3, 63, 9], [1, 26, 5], [10, 94, 13], [38, 53, 7], [31, 69, 12], [24, 70, 3]]
GREEDY SOLUTION FOUND: 
Remaining objects:[[24, 70, 3], [31, 69, 12], [17, 35, 10], [38, 53, 7], [37, 41, 11], [32, 3, 4]]
Solution:[[1, 26, 5], [3, 63, 9], [6, 99, 2], [10, 94, 10]]; V 1835; W 144


KeyboardInterrupt: 

<div class="alert alert-success" style="width:90%; margin:0 auto;">
<h2><p>BackTracking : 8 - Cartas</p></h2>
<p>
Supongamos que tenemos un conjunto de parejas de cartas del mismo valor. En particular, tenemos parejas de cartas hasta N. Por ejemplo, para $N=3$ tendríamos una pareja de 1s, una de 2s y una de 3s (un total de 6 cartas). Lo que queremos es encontrar la manera de ordenarlas para que entre las dos cartas de valor $n_i$ haya exactamente $n_i$ cartas. En el ejemplo con $N=3$ tendríamos la siguiente solución:
</p>
<center> <b>   3 - 1 - 2 - 1- 3 - 2 </b></center>
<p>
Podemos observar que entre los 3 hay tres cartas, entre los 2 hay dos y entre los 1 solo hay una. Dependiendo del tamaño de N el problema podría no tener solución. Por ejemplo $N=5$ o $N=6$. Se pide la implementación de un algoritmo que, mediante backtracking, devuelva una lista con la configuración encontrada dado un valor N. En caso de no haber solución, debe devolver una lista vacía.
</p> <p>
Es importante destacar que tendréis que prestar especial atención a la complejidad de vuestra solución, dado que el problema crece de forma no polinómica y para un valor $N>12$ podría tardar horas. Hay diferentes aproximaciones para resolverlo. Bien hecho, $N=12$ debería tardar escasos segundos. Si haces varias versiones, puedes entregarlas junto con la comparativa de complejidad correspondiente.</p>
</div>

In [23]:
'''
Implementa aquí tu solución. Debe devolver una lista con las cartas ordenadas tal y como se especifica
en el enunciado o None en caso de no existir solución para la entrada dada.
'''
def solve_deck(n, sol, placed):
    # If no more 0s in sol, we're done.
    if not sol.count(0):
        return sol,placed
    
    # Try placing numbers from bigger to smaller
    for i in range(n-1,-1,-1):
        # If i+1 hasn't been placed yet
        if not placed[i]:
            val=i+1                      # The actual number
            pos1=sol.index(0)            # First available position
            pos2=pos1+val+1              # Where should the next go
            # If next is within list and available
            if pos2<n*2 and sol[pos2]==0:
                # Place number in solution and keep going
                placed[i] = True         
                sol[pos1] = val
                sol[pos2] = val
                sol,placed=solve_deck(n,sol,placed)
                if not sol.count(0):
                    # If already finished, keep returning
                    return sol,placed
                else:
                    # Otherwise, backtrack
                    placed[i]=False
                    sol[pos1]=0
                    sol[pos2]=0
    return sol,placed

# Encontrado empiricamente
def has_sol(x):
    return (x % 4 == 0) or ((x+1) % 4 == 0)

def deck(n):
    # Initialise sol as a list of 0s
    if has_sol(n): return solve_deck(n,[0]*(2*n),[0]*n)[0]


In [24]:
for n in range(1,20):
    sol = deck(n)
    if sol:
        print("Sol "+str(n)+": ","-".join(map(str,sol)))
    else:
        print("Sol "+str(n)+": NO ENCONTRADA")

Sol 1: NO ENCONTRADA
Sol 2: NO ENCONTRADA
Sol 3:  3-1-2-1-3-2
Sol 4:  4-1-3-1-2-4-3-2
Sol 5: NO ENCONTRADA
Sol 6: NO ENCONTRADA
Sol 7:  7-4-1-5-1-6-4-3-7-5-2-3-6-2
Sol 8:  8-6-4-2-7-5-2-4-6-8-3-5-7-1-3-1
Sol 9: NO ENCONTRADA
Sol 10: NO ENCONTRADA
Sol 11:  11-9-7-5-10-2-6-8-2-5-7-9-11-6-4-10-8-3-1-4-1-3
Sol 12:  12-10-11-6-4-5-9-7-8-4-6-5-10-12-11-7-9-8-3-1-2-1-3-2
Sol 13: NO ENCONTRADA
Sol 14: NO ENCONTRADA
Sol 15:  15-13-14-10-8-6-4-2-11-12-2-4-6-8-10-13-15-14-9-7-11-3-12-5-1-3-1-7-9-5
Sol 16:  16-14-15-11-9-7-13-4-2-12-10-2-4-7-9-11-14-16-15-8-13-10-12-5-6-1-3-1-8-5-3-6
Sol 17: NO ENCONTRADA
Sol 18: NO ENCONTRADA
Sol 19:  19-17-18-14-12-16-9-7-5-15-2-11-13-2-5-7-9-12-14-17-19-18-16-11-10-15-13-8-3-4-6-1-3-1-4-10-8-6


<div class="alert alert-success" style="width:90%; margin:0 auto;">
<h2><p>Branch and bound : 9 - TSP</p></h2>
<p>

In [10]:
''
from utility import Node, PriorityQueue


def travel(adj_mat, src=0):
    optimal_tour = []
    n = len(adj_mat)
    if not n:
        raise ValueError("Invalid adj Matrix")
    u = Node()
    PQ = PriorityQueue()
    optimal_length = 0
    v = Node(level=0, path=[0])
    min_length = float('inf')  # infinity
    v.bound = bound(adj_mat, v)
    PQ.put(v)
    while not PQ.empty():
        v = PQ.get()
        if v.bound < min_length:
            u.level = v.level + 1
            for i in filter(lambda x: x not in v.path, range(1, n)):
                u.path = v.path[:]
                u.path.append(i)
                if u.level == n - 2:
                    l = set(range(1, n)) - set(u.path)
                    u.path.append(list(l)[0])
                    # putting the first vertex at last
                    u.path.append(0)

                    _len = length(adj_mat, u)
                    if _len < min_length:
                        min_length = _len
                        optimal_length = _len
                        optimal_tour = u.path[:]

                else:
                    u.bound = bound(adj_mat, u)
                    if u.bound < min_length:
                        PQ.put(u)
                # make a new node at each iteration! python it is!!
                u = Node(level=u.level)

    # shifting to proper source(start of path)
    optimal_tour_src = optimal_tour
    if src is not 1:
        optimal_tour_src = optimal_tour[:-1]
        y = optimal_tour_src.index(src)
        optimal_tour_src = optimal_tour_src[y:] + optimal_tour_src[:y]
        optimal_tour_src.append(optimal_tour_src[0])

    return optimal_tour_src, optimal_length


def length(adj_mat, node):
    tour = node.path
    # returns the sum of two consecutive elements of tour in adj[i][j]
    return sum([adj_mat[tour[i]][tour[i + 1]] for i in xrange(len(tour) - 1)])


def bound(adj_mat, node):
    path = node.path
    _bound = 0

    n = len(adj_mat)
    determined, last = path[:-1], path[-1]
    # remain is index based
    remain = filter(lambda x: x not in path, range(n))

    # for the edges that are certain
    for i in xrange(len(path) - 1):
        _bound += adj_mat[path[i]][path[i + 1]]

    # for the last item
    _bound += min([adj_mat[last][i] for i in remain])

    p = [path[0]] + remain
    # for the undetermined nodes
    for r in remain:
        _bound += min([adj_mat[r][i] for i in filter(lambda x: x != r, p)])
    return _bound


if __name__ == '__main__':

    matrix = [
        [0, 14, 4, 10, 20],
        [14, 0, 7, 8, 7],
        [4, 5, 0, 7, 16],
        [11, 7, 9, 0, 2],
        [18, 7, 17, 4, 0]
    ]
    print travel(matrix)

SyntaxError: invalid syntax (<ipython-input-10-7e29c11fb0bf>, line 92)